In [88]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from bs4 import BeautifulSoup
from openai_interface import GPT_Turbo
import re
import chromadb

ImportError: cannot import name 'BaseSettings' from 'pydantic' (/anaconda/envs/openai/lib/python3.9/site-packages/pydantic/__init__.cpython-39-x86_64-linux-gnu.so)

In [8]:
def sub_new_lines(content: str):
    return re.sub('\n+', '\n', content)

In [82]:
def get_webpage(url: str, return_content_only: bool=False, return_as_loader: bool=False):
    if return_as_loader:
        return WebBaseLoader(url)
    data = WebBaseLoader(url).load()
    if return_content_only:
        content = data[0].page_content.strip()
        return sub_new_lines(content)
    else: return data

In [10]:
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
langurl = "https://python.langchain.com/docs/use_cases/question_answering/"

In [84]:
page = get_webpage(langurl)
loader = get_webpage(langurl, return_as_loader=True)
loader

In [89]:
index = VectorstoreIndexCreator().from_loaders([loader])

ValueError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [20]:
gpt = GPT_Turbo()

In [55]:
resume = '''
EXECUTIVE SUMMARY

Former US Navy SEAL and hands-on leader with an eclectic background that combines real-world military operational excellence with six years of corporate sales, strategy, and Data Science experience. Big thinker with proven ability to translate data insights into business decisions; years of experience improving operational processes. 

PROFESSIONAL EXPERIENCE
Microsoft – Senior Data Scientist 	Apr 2021 – Present
•  Recently assigned as the Lead Product data scientist for the Azure Planetary Computer product team. 
•  Technical Lead for 5-person team charged with creating a search and discovery experience for enterprise data   
   within Classified networks, consisting of keyword/semantic/image/video search, and question answering.
•  Lead Delivery data scientist for a strategic military account; responsible for scoping and implementing a year-
   long contractual engagement that resulted in two spin-off product applications. 

Elite Meet – Head of Analytics 	 Oct 2020 – June 2021
•  Built an analytics platform for the CEO to enable decision support in the areas of membership growth,  
    marketing campaigns, and resource allocation by geographic location.   

Gartner – Federal Account Executive	      2019 – Oct 2020
•  Achieved over 200% of new business quota in nine months by direct engagement with CXO decision makers.

US Navy: Naval Special Warfare Command
Product Manager	             		    	     	   						    2017
•  Led the design, development, implementation and user experience of flagship, classified software platform.
•  Grew the daily user base by 400% and increased adoption rate from three organizations to 17 over 7 months.

Chief of Staff				                                           				        2016-2017
•  High impact role leading a staff of 13 departments in support of the Logistics requirements of the West Coast   
   SEAL Teams and seven other supporting commands.
•  Planned and executed an extensive 6-months long Manpower Requirements project involving a detailed 
   analysis of all organizational commitments which was used to justify organizational manpower requirements 
   into 2020.

Director of Operations							                                            2015-2016
•  Managed the training, equipping, and deployment of over 450 combat support personnel, including six 
   specialized Combat Support Teams, across 4 continents, in support of SEAL Team combat operations. 

Senior Program Manager	       								        2010-2015
•  Spearheaded the relocation of headquarters building from Virginia to Florida. Over a period of 2 years, I worked with 
   finance, manpower, operations and logistics to create a framework for project execution, resulting in the transfer of over  
   45 personnel and $30M in assets. 
•  Hand-picked to lead a SEAL Task Unit charged with augmenting the security of the President of the United States 
   during a 2-week period while he conducted a high-level diplomatic visit to South Africa. 

EDUCATION
UNIVERSITY OF CALIFORNIA, BERKELEY | Masters, Data Science				    	        Aug 2019
UNIVERSITY OF UTAH | BS, Political Science (Honors), Magna Cum Laude				        May 2007

'''

In [28]:
prompt = f'''
# Using the enclosed context in triple backticks answer the following question:
# Question: Which method or Class should I use to parse text content from a web page and return in string format, using the python library langchain?  Provide a step by step solution on how I would do that using python.
# Context: 
# ```
# {page}
# ```
# Answer: 
'''

In [90]:
prompt = f'''
What is the total number of people that Chris Sanchez has led or managed over the entire course of his career. \
Provide a rough estimate, and reason through the following context to answer the question.
Context: {resume}
'''

In [91]:
response = gpt.get_completion_from_messages(prompt, show_response=False)

In [92]:
print(response)

Based on the provided context, it is difficult to determine the exact number of people that Chris Sanchez has led or managed over the entire course of his career. However, we can make a rough estimate based on the information provided.

From the context, we can identify the following roles where Chris Sanchez had leadership or management responsibilities:

1. Lead Product data scientist at Microsoft: Although the number of people directly managed is not mentioned, it can be assumed that Chris Sanchez is leading a team of data scientists, which could range from a few individuals to a larger team.

2. Head of Analytics at Elite Meet: The context does not provide information about the size of the team or the number of people managed in this role.

3. Federal Account Executive at Gartner: The context does not mention any direct management responsibilities in this role.

4. Product Manager at US Navy: Naval Special Warfare Command: The context does not provide information about the size of 

In [40]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

In [78]:
splits = splitter.split_documents(page)